In [6]:
import OffsetArrays
using Plots
using Revise
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using IterTools
using DelimitedFiles
include("../src/bp.jl") 
include("../src/post_infer.jl") 
include("../src/observables.jl") #utils functions for calculating observables and saving them

inf_vs_dil_optimal (generic function with 1 method)

# Population dynamics for the messages

In [7]:
# S=0   I=1
# The times go from ti = 0 to ti=T+1
# when ti=0 it means that i is the zero patient
# when ti=T+1 it means that i is never infected

T = 8 # discrete time
N = 10000; #population size
fr = 0.0 # noise of measure
grid = 21 # the grid of the phase diagram
dilution = 0.0

λRange = LinRange(0.05,0.9,grid) 
γRange = LinRange(0.05,0.9,grid) 

21-element LinRange{Float64, Int64}:
 0.05,0.0925,0.135,0.1775,0.22,0.2625,…,0.6875,0.73,0.7725,0.815,0.8575,0.9

In [ ]:
#here we choose what regime we want to study (λ VS dilution, λ VS γ, dilution VS fr ...)
degree_dist = Poisson(3)
γi = 0.5
λi = 0.5
inf_out = inf_vs_gam_learn(γRange, λRange, γi, λi, N, T, degree_dist, fr , dilution, tot_iterations = 100 )

In [ ]:
# Here we write the results in files.
samp = 11
graph = "er"

In [ ]:
open("./dil_lam_$(graph)_$(samp)/ninf.txt","w") do io
    writedlm(io,inf_out[:,:,1]) 
end
for t = 1:T+1
    open("./dil_lam_$(graph)_$(samp)/AUC$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/Ov$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + T + 1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/L1_$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 2*T + 2]) 
    end
    open("./dil_lam_$(graph)_$(samp)/MSE$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 3*T + 3]) 
    end
end
open("./dil_lam_$(graph)_$(samp)/Fe.txt","w") do io
    writedlm(io,inf_out[:,:,end]) 
end